<a href="https://colab.research.google.com/github/F1ame21/Primat_Labs/blob/main/lab3/LU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math
import copy
import time
from numpy import linalg as LA
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse import rand

In [2]:
data = np.array([9, 3, 1, 1, 11, 2, 1, 2, 1, 10, 2, 2, 1, 
                 2, 9, 1, 1, 1, 12, 1, 8, 2, 2, 3, 8])
indices = np.array([0, 3, 4, 6, 1, 2, 3, 6, 1, 2, 3, 0, 1, 2, 
                    3, 4, 0, 3, 4, 6, 5, 0, 1, 4, 6])
indptr = np.array([0, 4, 8, 11, 16, 20, 21, 25])
#sr_matrix((data, indices , indptr), shape=(7, 7)).toarray()

In [3]:
def empty_matrix(n, m, format):
    """Пустой нулевой двумерный массив
    :param n: количество строк
    :param m: количество столбцов
    :return: python array
    """
    Matrix = scipy.sparse.__dict__[format + "_matrix"]
    return Matrix((n, m))

In [4]:
def generate_diagonal_domination_matrix(a, k):
  """
  Генерирует тестовое уравнение для решения
  :param a: квадратная матрица коэффициентов a_ij (см. лабу, а также отчет)
  :param k: номер уравнения в последовательности
  :return: пара (A_k, F_k) для уравнения A_k * x_k = F_k
  None - если уравнение несовместно
    """
  n = a.shape[0]
  A_k = empty_matrix(n, n, "lil")
  for i in range(n):
    s1 = -sum(a[i, q] for q in range(0, i))
    s2 = -sum(a[i, q] for q in range(i+1, n))
    s = s1 + s2
    for j in range(n):
      if i == j:
        A_k[i, j] = s + pow(10, -k)
      else:
        A_k[i, j] = a[i, j]
  A_k = A_k.tocsr()
  return A_k
      

In [5]:
def generate_hilbert_matrix(k):
    """Генерирует тестовое уравнение для решения с матрицей Гильберта
    :param k: номер уравнения в последовательности
    :return: пара (A_k, F_k) для уравнения A_k * x_k = F_k
    None - если уравнение несовместно
    """
    A_k = empty_matrix(k, k, "lil")
    for i in range(k):
        for j in range(k):
            A_k[i, j] = 1.0 / (i + j + 1.0)

    return A_k.tocsr()


In [6]:
# из рязряженной - в матрицу
def matrix_conversion(data, ind, indptr):
  n = len(indptr)-1
  A = np.zeros([n, n])
  for i in range(0, n):
    k = indptr[i+1] - indptr[i]
    for j in range(0, k):
      A[i, ind[0]] = data[0]
      data = np.delete(data, 0)
      ind = np.delete(ind, 0)
  return A

In [7]:
# LU - разложение, матрицы А
def LU(a):
  n = a.shape[0]
  matrix_L = np.eye(n)
  matrix_U = np.zeros([n, n])
  for i in range(0, n):
    for j in range(0, n):
      if i <= j:
        s = sum(matrix_U[k, j] * matrix_L[i, k] for k in range(0, i))
        matrix_U[i, j] = a[i, j] - s
      else:
        s = sum(matrix_U[k, j] * matrix_L[i, k] for k in range(0, j))
        matrix_L[i, j] = (a[i, j] - s) / matrix_U[j, j]
  return matrix_L, matrix_U

In [8]:
# Нахождение обратной матрицы L
def matrix_L_inv(L):
  n = L.shape[0]
  L_inv = np.zeros([n, n])
  for i in range(0, n):
    for j in range(0, n):
      if i == j:
        L_inv[i, j] = 1 / L[i, j]
      elif i > j:
        s = sum(L_inv[k, j] * L[i, k] for k in range(0, i))
        L_inv[i, j] = (-1 * s)/ L[i, i]
  return L_inv
# Нахождение обратной матрицы U
def matrix_U_inv(U):
  n = len(U)
  U_inv = np.zeros([n, n])
  for i in range(0, n):
    for j in range(0, n):
      if i == j:
        U_inv[i, j] = 1 / U[i, j]
      else:
         s = sum(U_inv[i, k] * U[k, j] for k in range(0, j))
         U_inv[i, j] = (-1 * s) / U[j, j]
  return U_inv

In [9]:
# Решение Ly = b
def solve_Ly(L, b):
  n = L.shape[0]
  y = np.zeros([n, 1])
  for i in range(0, n):
    s = sum(L[i, k] * y[k] for k in range(0, i))
    y[i] = b[i] - s
  return y

# Решение Ux = y
def solve_Lx(U, y):
  n = len(U)
  x = np.zeros([n, 1])
  for i in reversed(range(0, n)):
    s = sum(U[i, k] * x[k] for k in range(i, n))
    x[i, 0] = (y[i, 0] - s) / U[i, i]
  return x


In [10]:
# Проверка на корректность матрицы
def isCorrectAway(A):
  for i in range(len(A)):
    if (A[i, i] == 0):
      print('На главной диагонале присутсуют нули')
      return False
  return True
# Функция проверки
def isNeedToComplete(x_old, x_new, eps = 0.00001, sum1 = 0, sum2 = 0):
  for i in range(0, len(x_old)):
    sum1 += (x_new[i] - x_old[i]) ** 2
    sum2 += x_new[i] ** 2
  return math.sqrt( sum1 / sum2 ) < eps 

# Процедура решения
# count - размерность; 
# Max_Iter - максимальное количетсво итераций; 
# numberOfIter- номер итерации;
def solutin(a, b, Max_Iter = 200000, numberOfIter = 0):
  count = a.shape[0]
  x = np.zeros([count, 1])
  start = time()
  while numberOfIter < Max_Iter:
    x_prev = copy.deepcopy(x)
    for k in range(0, count):
      s = 0
      for j in range(0, count):
        if (j != k):
          s += a[k, j] * x[j]
      x[k] = b[k, 0]/a[k, k] - s/a[k, k]
    if isNeedToComplete(x_prev, x):
      break
    numberOfIter +=1
  stop = time()
  return x, numberOfIter, stop - start

In [11]:
def error(a, x, b):
  b2 = np.matmul(a, x)
  error = abs(b) - abs(b2)
  return error

In [12]:
def generate_big_matrix(n, p, format):
    """Генерирует большую матрицу nxn разреженности p
    :param n: размерность или порядок матрицы
    :param p: отношение ненулевых клеток к nxn
    :return: матрица A в разреженном виде
    """
    return scipy.sparse.random(n, n, p, format=format)

In [13]:
def LU_solve(A):
  start = time()
  matrix_L, matrix_U = LU(A)
  y = solve_Ly(matrix_L, b)
  x = solve_Lx(matrix_U, y)
  stop = time()
  return x, stop - start

In [14]:
from time import time

In [27]:
if __name__ == '__main__':
  #A = matrix_conversion(data, indices, indptr)
  #A = np.random.randint(-10, 10, (10, 10))
  A1 = generate_big_matrix(500, 0.0000001, 'lil')
  #A = np.array([[500.000001, -3., -2.],
  #              [-2., 400.000001, -2.],
  #              [-1., -2., -300.000001]])
  b = np.random.randint(-100, 100, (500, 1))
  #b = np.array([[-3], [-4], [-2], [-1]]
  A = generate_diagonal_domination_matrix(A1, 1)

  #A = generate_hilbert_matrix(4).toarray()
  n = A.shape[0]

  #L_inv = matrix_L_inv(matrix_L)
  #U_inv = matrix_U_inv(matrix_U)
  #matrix_A_inv = np.dot(U_inv, L_inv)
  #x = LU_solve(A)[0]]
  print(solutin(A, b)[2])
  print(LU_solve(A)[1])
  #print(LU_solve(A)[1])
  #jacobi = solutin(A, b)
  #print(solutin(A, b)[2])
  #print(
  #    f'Матрица A:\n{A}\n\nМатрица b:\n{2}\n\n Решение Ax=b \n с использование LU-разложения\n{x}\n\nПотребовалось: {toc1-tic1} времени\n\nПогрешность:\n{1}\n\nМатрица L:\n{matrix_L}\n\nМатрица U:\n{matrix_U}\n\n',
      #f'Обратная матрица:\n{matrix_A_inv}\n\n',
  #    f'Решение методом Якоби\n{jacobi[0]}\n\n'
  #    f'Потребовалось: {jacobi[2]} времени\n\n'
  #    f'Потребовалось: {jacobi[1]} шагов\n\n'
  #    f'Погрешность:\n{1}'
  #   )

13.718498945236206
33.53236937522888
